In [7]:
import pandas as pd
import numpy as np

In [8]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml("mnist_784", version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [9]:
X, y = mnist['data'], mnist['target']
print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


In [10]:
X.head() # 28*28 像素

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
y.head()

0    5
1    0
2    4
3    1
4    9
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [12]:
X_train, X_test, y_train, y_test = X[:60000].values, X[60000:].values, y[:60000].values, y[60000:].values

In [13]:
X_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## 训练模型  

逻辑回归 + 随机梯度下降

loss function可以通过loss参数进行设置。SGDClassifier支持下面的loss函数：  
loss=”hinge”: (soft-margin)线性SVM.  
loss=”modified_huber”: 带平滑的hinge loss.  
loss=”log_loss”: logistic回归  

In [9]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(loss='log_loss', max_iter=1000, random_state=42)
sgd_clf.fit(X_train, y_train)

SGDClassifier(loss='log_loss', random_state=42)

In [10]:
y_pred = sgd_clf.predict(X_test)
y_pred

array(['7', '2', '1', ..., '4', '5', '6'], dtype='<U1')

网格搜索优化超参数

loss: 指定损失函数。常用的选项包括：

'hinge': 支持向量机（SVM）损失（默认值）。
'log_loss': 逻辑回归损失。
'modified_huber': 平滑的支持向量机损失。
'squared_hinge': 平方的支持向量机损失。
'perceptron': 感知机损失。
'epsilon_insensitive': 支持向量回归（SVR）损失。
'squared_epsilon_insensitive': 平方的支持向量回归损失。
penalty: 指定正则化项。常用的选项包括：

'l2': L2 正则化（默认值）。
'l1': L1 正则化。
'elasticnet': 弹性网络正则化（L1 和 L2 的组合）。
alpha: 正则化项的常数，默认为 0.0001。值越大，正则化效果越强。

l1_ratio: 当 penalty='elasticnet' 时，指定 L1 和 L2 正则化的比例。范围在 [0, 1] 之间。

fit_intercept: 是否计算截距项。默认为 True。

max_iter: 最大迭代次数。默认为 1000。

tol: 优化的容忍度。默认为 1e-3。

shuffle: 是否在每次迭代前打乱数据。默认为 True。

verbose: 控制训练过程的详细程度。默认为 0（不输出）。

epsilon: 当 loss='epsilon_insensitive' 或 loss='squared_epsilon_insensitive' 时，指定 epsilon 的值。默认为 0.1。

n_jobs: 并行计算时使用的 CPU 核心数。默认为 None（使用单个核心）。

random_state: 随机数种子，用于打乱数据和初始化权重。

learning_rate: 学习率调度。常用的选项包括：

'constant': 固定学习率。
'optimal': 根据训练数据自动调整学习率。
'invscaling': 逆比例缩放学习率。
'adaptive': 自适应学习率。
eta0: 初始学习率。默认为 0.0。

power_t: 当 learning_rate='invscaling' 时，指定学习率的指数。默认为 0.5。

early_stopping: 是否使用早停法来终止训练。默认为 False。

validation_fraction: 当 early_stopping=True 时，指定用于验证的训练数据比例。默认为 0.1。

n_iter_no_change: 当 early_stopping=True 时，指定在多少次迭代后没有改进就停止训练。默认为 5。

class_weight: 类别权重。可以是 'balanced' 或一个字典，默认为 None。

warm_start: 是否使用上一次调用的解决方案来初始化。默认为 False。

average: 是否使用平均 SGD。默认为 False。

In [25]:
from sklearn.model_selection import GridSearchCV


params_grid = {
    'loss': ['log_loss'],
    'penalty': ['elasticnet'],
    'l1_ratio': np.arange(0.2, 0.9, 0.1),
    'max_iter': [1000],
    'random_state': [42],
    'n_jobs': [-1]
}

grid = GridSearchCV(sgd_clf, params_grid, cv=5, scoring='f1_micro', n_jobs=-1)

In [26]:
grid.fit(X_train, y_train)

In [14]:
grid.best_params_

{'l1_ratio': 0.2,
 'learning_rate': 'optimal',
 'loss': 'log_loss',
 'max_iter': 1000,
 'n_jobs': -1,
 'penalty': 'elasticnet',
 'random_state': 42}

In [15]:
sgd_clf_best = grid.best_estimator_

In [14]:
sgd_clf_best.fit(X_train, y_train)
y_pred = sgd_clf_best.predict(X_test)

In [15]:
from sklearn.metrics import f1_score


weighted_f1 = f1_score(y_test, y_pred, average='weighted')
weighted_f1

0.8633743117809816

模型持久化

In [16]:
import joblib

pkl_path = 'model/sgd_clf.pkl'

joblib.dump(sgd_clf_best, pkl_path)

['model/sgd_clf.pkl']

In [17]:
sgd_clf_best = joblib.load(pkl_path)